In [56]:
from algosdk.v2client import algod
import json

In [57]:
algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = ""
headers = {
    "X-API-Key": "ujczQlZdDQ8aIkHGGDjwf1NAAnZHmHej7Orb1Llt",
}

In [58]:
algod_client = algod.AlgodClient(algod_token, algod_address, headers)

try:
    status = algod_client.status()
    print("Status: " + json.dumps(status, indent=2, sort_keys=True))
except Exception as e:
    print("Failed to get algod status: {}".format(e))

Status: {
  "catchpoint": "",
  "catchpoint-acquired-blocks": 0,
  "catchpoint-processed-accounts": 0,
  "catchpoint-processed-kvs": 0,
  "catchpoint-total-accounts": 0,
  "catchpoint-total-blocks": 0,
  "catchpoint-total-kvs": 0,
  "catchpoint-verified-accounts": 0,
  "catchpoint-verified-kvs": 0,
  "catchup-time": 0,
  "last-catchpoint": "27400000#RARNY2TB3EF2Z2FDFQXRTDWYPTPU5D36RQK2QNLQ2JPERPTQF46A",
  "last-round": 27403034,
  "last-version": "https://github.com/algorandfoundation/specs/tree/44fa607d6051730f5264526bf3c108d51f0eadb6",
  "next-version": "https://github.com/algorandfoundation/specs/tree/44fa607d6051730f5264526bf3c108d51f0eadb6",
  "next-version-round": 27403035,
  "next-version-supported": true,
  "stopped-at-unsupported-round": false,
  "time-since-last-round": 1886036855
}


In [59]:
# Retrieve latest block information                                                                                                                                               
last_round = status.get("last-round")
print("####################")
try:
    block = algod_client.block_info(last_round)
    # print("Latest block: " + json.dumps(block, indent=2, sort_keys=True))
except Exception as e:
    print("Failed to get algod status: {}".format(e))

####################


In [60]:
from algosdk import account, encoding

# generate an account
private_key, address = account.generate_account()
print("Private key:", private_key)
print("Address:", address)

# check if the address is valid
if encoding.is_valid_address(address):
    print("The address is valid!")
else:
    print("The address is invalid.")

Private key: OtPn16hfvs2lTd870OYG5Qfgr0RznyLyUuLQxi5YUH/t/vt3nbEn9XQA8USvUuOhlRRiMTqJiAfo/58t6hOPdw==
Address: 5X7PW545WET7K5AA6FCK6UXDUGKRIYRRHKEYQB7I76PS32QTR53YTWODII
The address is valid!


In [61]:

algod_client.account_info(address)
#  send_transaction()

{'address': '5X7PW545WET7K5AA6FCK6UXDUGKRIYRRHKEYQB7I76PS32QTR53YTWODII',
 'amount': 0,
 'amount-without-pending-rewards': 0,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0},
 'assets': [],
 'created-apps': [],
 'created-assets': [],
 'min-balance': 100000,
 'pending-rewards': 0,
 'reward-base': 27521,
 'rewards': 0,
 'round': 27403034,
 'status': 'Offline',
 'total-apps-opted-in': 0,
 'total-assets-opted-in': 0,
 'total-created-apps': 0,
 'total-created-assets': 0}

## Transaction

In [62]:
import json
import time
import base64
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction

In [63]:
# Function from Algorand Inc.
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print('Waiting for confirmation')
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print('Transaction confirmed in round', txinfo.get('confirmed-round'))
    return txinfo

In [64]:
# # Setup HTTP client w/guest key provided by PureStake
# algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
# algod_token = ""
# headers = {
#     "X-API-Key": "B3SU4KcVKi94Jap2VXkK83xx38bsv95K5UZm2lab",
# }

In [65]:
# Initalize throw-away account for this example - check that is has funds before running script
mnemonic_phrase = 'athlete inquiry intact erupt dog poverty sentence quick ask laundry sugar helmet glow intact green blush swift youth crash ahead echo urge quality abandon absent';
account_private_key = mnemonic.to_private_key(mnemonic_phrase)
account_public_key = mnemonic.to_public_key(mnemonic_phrase)

# algodclient = algod.AlgodClient(algod_token, algod_address, headers)

In [66]:
algod_client.account_info(account_public_key)

{'address': 'QECL3HMPEG4XWX5IKN2JNR3W5TW5EKTFJGE4UWGDGRIGW44ICCWMNNBQIE',
 'amount': 5000000,
 'amount-without-pending-rewards': 5000000,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0},
 'assets': [],
 'created-apps': [],
 'created-assets': [],
 'min-balance': 100000,
 'pending-rewards': 0,
 'reward-base': 27521,
 'rewards': 0,
 'round': 27403035,
 'status': 'Offline',
 'total-apps-opted-in': 0,
 'total-assets-opted-in': 0,
 'total-created-apps': 0,
 'total-created-assets': 0}

In [67]:
# get suggested parameters from Algod
params = algod_client.suggested_params()

gh = params.gh
first_valid_round = params.first
last_valid_round = params.last
fee = params.min_fee
send_amount = 10

existing_account = account_public_key
send_to_address = 'YKAWZVFGUVQCUERS5GCGKEQTHYFA5RYWBVDUCLLG623BJ2C7ACFPMA6CD'


In [68]:
"""# Create and sign transaction
tx = transaction.PaymentTxn(existing_account, fee, first_valid_round, last_valid_round, gh, send_to_address,
                            send_amount, flat_fee=True)
signed_tx = tx.sign(account_private_key)

try:
    tx_confirm = algod_client.send_transaction(signed_tx)
    print('Transaction sent with ID', signed_tx.transaction.get_txid())
    wait_for_confirmation(algod_client, txid=signed_tx.transaction.get_txid())
except Exception as e:
    print(e)"""

"# Create and sign transaction\ntx = transaction.PaymentTxn(existing_account, fee, first_valid_round, last_valid_round, gh, send_to_address,\n                            send_amount, flat_fee=True)\nsigned_tx = tx.sign(account_private_key)\n\ntry:\n    tx_confirm = algod_client.send_transaction(signed_tx)\n    print('Transaction sent with ID', signed_tx.transaction.get_txid())\n    wait_for_confirmation(algod_client, txid=signed_tx.transaction.get_txid())\nexcept Exception as e:\n    print(e)"

In [69]:
import hashlib

def get_hash(img_path):
    # This function will return the `md5` checksum for any input image.
    with open(img_path, "rb") as f:
        img_hash = hashlib.md5()
        while chunk := f.read(8192):
           img_hash.update(chunk)
    return img_hash.hexdigest()

In [70]:


fullPathImage = './images/task2.jpg'
imageIntegrity = "sha256-" + get_hash(fullPathImage)

# metadatafileImage = (await fs.readFileSync(fullPathImage))
# metadatafileImage = (await fs.readFile(fullPathImage))
# hashImage = crypto.createHash('sha256')
# hashImage.update(metadatafileImage)
# hashImageBase64 = hashImage.digest("base64")
# imageIntegrity = "sha256-" + hashImageBase64

# print("image_integrity : " + imageIntegrity)

In [71]:
import json
# dir_path = os.path.dirname(os.path.realpath(__file__))
f = open ( './backend/trainee/nft/metadata.json', "r")



# Reading from file
metadataJSON = json.loads(f.read())
metadataJSON['image_integrity']=imageIntegrity
metadataStr = json.dumps(metadataJSON)

In [72]:
metadataStr

'{"name": "Trainee Certificate", "description": "Certificate", "image_integrity": "sha256-87f8aa8bd5fd1263e23e3069243dad62", "properties": {"simple_property": "Trainee unique Certificate", "rich_property": {"name": "Certificate", "value": "001", "display_value": "001", "class": "emphasis", "css": {"color": "#ffffff", "font-weight": "bold", "text-decoration": "underline"}}, "array_property": {"name": "Certificate", "value": [1, 2, 3, 4], "class": "emphasis"}}}'

Create assets 

In [73]:
import json
import hashlib
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, wait_for_confirmation

In [74]:
algod_token = "2f3203f21e738a1de6110eba6984f9d03e5a95d7a577b34616854064cf2c0e7b"
algod_address = "https://academy-algod.dev.aws.algodev.network/"
algod_client = algod.AlgodClient(algod_token, algod_address)

In [75]:
def print_created_asset(algodclient, account, assetid):
  # note: if you have an indexer instance available it is easier to just use this
  # response = myindexer.accounts(asset_id = assetid)
  # then use 'account_info['created-assets'][0] to get info on the created asset
  account_info = algodclient.account_info(account)
  idx = 0
  for my_account_info in account_info['created-assets']:
    scrutinized_asset = account_info['created-assets'][idx]
    idx = idx + 1       
    if (scrutinized_asset['index'] == assetid):
      print("Asset ID: {}".format(scrutinized_asset['index']))
      print(json.dumps(my_account_info['params'], indent=4))
      break

#   Utility function used to print asset holding for account and assetid
def print_asset_holding(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then loop thru the accounts returned and match the account you are looking for
    account_info = algodclient.account_info(account)
    idx = 0
    for my_account_info in account_info['assets']:
        scrutinized_asset = account_info['assets'][idx]
        idx = idx + 1        
        if (scrutinized_asset['asset-id'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['asset-id']))
            print(json.dumps(scrutinized_asset, indent=4))
            break

In [80]:
# accepts the mnemonic
def create_asset(m,receiver_pk):  
  accounts = {}
  accounts[1] = {}
  accounts[1]['pk'] = mnemonic.to_public_key(m)
  accounts[1]['sk'] = mnemonic.to_private_key(m)

  print("--------------------------------------------")
  print("Creating Asset...")
  # CREATE ASSET
  # Get network params for transactions before every transaction.
  params = algod_client.suggested_params()
  # comment these two lines if you want to use suggested params
  # params.fee = 1000
  # params.flat_fee = True
    
  # JSON file
#   dir_path = os.path.dirname(os.path.realpath(__file__))
#   f = open (dir_path + '/NFT/metadata.json', "r")
  f = open ('./backend/trainee/nft/metadata.json', "r")


  
  # Reading from file
  metadataJSON = json.loads(f.read())
  metadataStr = json.dumps(metadataJSON)

  hash = hashlib.new("sha512_256")
  hash.update(b"arc0003/amj")
  hash.update(metadataStr.encode("utf-8"))
  json_metadata_hash = hash.digest()


  # Account 1 creates an asset called latinum and
  # sets Account 1 as the manager, reserve, freeze, and clawback address.
  # Asset Creation transaction
  txn = AssetConfigTxn(
      sender=accounts[1]['pk'],
      sp=params,
      total=1,
      default_frozen=False,
      unit_name="H001",
      asset_name="Harriet's Artwork 001",
      manager=accounts[1]['pk'],
      reserve=receiver_pk,
      freeze=None,
      clawback=None,
      strict_empty_address_check=False,
      url="https://gateway.pinata.cloud/ipfs/QmXARven5z9VC64kt8ZHwA6R3qT3bfqXkJftJRHZtrNMye", 
      metadata_hash=json_metadata_hash,
      decimals=0)

  # Sign with secret key of creator
  stxn = txn.sign(accounts[1]['sk'])

  # Send the transaction to the network and retrieve the txid.
  txid = algod_client.send_transaction(stxn)
  print("Asset Creation Transaction ID: {}".format(txid))


  # Wait for the transaction to be confirmed
  confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
  print("TXID: ", txid)
  print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
  try:
      # Pull account info for the creator
      # account_info = algod_client.account_info(accounts[1]['pk'])
      # get asset_id from tx
      # Get the new asset's information from the creator account
      ptx = algod_client.pending_transaction_info(txid)
      asset_id = ptx["asset-index"]
      print_created_asset(algod_client, accounts[1]['pk'], asset_id)
      print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
  except Exception as e:
      print(e)


In [77]:
# OPT-IN
def opt_in(trainee_m,asset_id):

    # Check if asset_id is in account 3's asset holdings prior
    # to opt-in
    trainee_sk=mnemonic.to_private_key(trainee_m)
    trainee_pk=mnemonic.to_public_key(trainee_m)
    params = algod_client.suggested_params()
    # comment these two lines if you want to use suggested params
    params.fee = 1000
    params.flat_fee = True


    # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = AssetTransferTxn(
        sender=trainee_pk,
        sp=params,
        receiver=trainee_pk,
        amt=0,
        index=asset_id)
    stxn = txn.sign(trainee_sk)
    txid = algod_client.send_transaction(stxn)
    print(txid)
    # Wait for the transaction to be confirmed
    wait_for_confirmation(algod_client, txid)
    # Now check the asset holding for that account.
    # This should now show a holding with a balance of 0.
    print_asset_holding(algod_client, trainee_pk, asset_id)

In [78]:
# TRANSFER ASSET
def transfer_asset(sender_m,receiver_pk,asset_id):

    sender_pk=mnemonic.to_public_key(sender_m)
    sender_sk=mnemonic.to_private_key(sender_m)

    # transfer asset of 10 from account 1 to account 3
    params = algod_client.suggested_params()
    # comment these two lines if you want to use suggested params
    params.fee = 1000
    params.flat_fee = True
    txn = AssetTransferTxn(
        sender=sender_pk,
        sp=params,
        receiver=receiver_pk,
        amt=1,
        index=asset_id)
    stxn = txn.sign(sender_sk)
    txid = algod_client.send_transaction(stxn)
    print(txid)
    # Wait for the transaction to be confirmed
    wait_for_confirmation(algod_client, txid)
    # The balance should now be 10.
    print_asset_holding(algod_client, receiver_pk, asset_id)

In [81]:
# Sender 
import os,sys
cwd = os.getcwd()
sys.path.append(f'./backend/trainee')

def create_account(fund=True):
  # Change algod_token and algod_address to connect to a different client
  algod_token = "2f3203f21e738a1de6110eba6984f9d03e5a95d7a577b34616854064cf2c0e7b"
  algod_address = "https://academy-algod.dev.aws.algodev.network/"
  algod_client = algod.AlgodClient(algod_token, algod_address)

  #Generate new account for this transaction
  secret_key, my_address = account.generate_account()
  m = mnemonic.from_private_key(secret_key)
  print("My address: {}".format(my_address))

  # Check your balance. It should be 0 microAlgos
  account_info = algod_client.account_info(my_address)
  print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

  if fund:
    #Fund the created account
    print('Go to the below link to fund the created account using testnet faucet: \n https://dispenser.testnet.aws.algodev.network/?account={}'.format(my_address)) 

    completed = ""
    while completed.lower() != 'yes':
      completed = input("Type 'yes' once you funded the account: ")

    print('Fund transfer in process...')
    # Wait for the faucet to transfer funds
    time.sleep(5)
    
    print('Fund transferred!')
    # Check your balance. It should be 5000000 microAlgos
    account_info = algod_client.account_info(my_address)
    print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

  return m

m=create_account()

My address: OSGKCHOFABX4O3RCBY7SBH4ECDVWSV2JLVRQDZM72HBG5TBMWMXBKVJEOM
Account balance: 0 microAlgos

Go to the below link to fund the created account using testnet faucet: 
 https://dispenser.testnet.aws.algodev.network/?account=OSGKCHOFABX4O3RCBY7SBH4ECDVWSV2JLVRQDZM72HBG5TBMWMXBKVJEOM
Fund transfer in process...
Fund transferred!
Account balance: 5000000 microAlgos



In [87]:
# receiver 
receiver=create_account()
#algod_client.account_info(address="2MMN5ZIVR3NGNXLT25Q6ZWJGAXBREG75DTFN5KYJNMSFU7RP2NFB3BQ5KA")

My address: 2UZFZ27QS7VTVAAJ77UTCB7S5JKA6YWAICLKSPRIUULK7U2IRSNCN3VVA4
Account balance: 0 microAlgos

Go to the below link to fund the created account using testnet faucet: 
 https://dispenser.testnet.aws.algodev.network/?account=2UZFZ27QS7VTVAAJ77UTCB7S5JKA6YWAICLKSPRIUULK7U2IRSNCN3VVA4
Fund transfer in process...
Fund transferred!
Account balance: 5000000 microAlgos



In [99]:
receiver_pk="2UZFZ27QS7VTVAAJ77UTCB7S5JKA6YWAICLKSPRIUULK7U2IRSNCN3VVA4"

In [101]:
account_info = algod_client.account_info(receiver_pk)
# holding = None
# idx = 0
# for my_account_info in account_info['assets']:
#     scrutinized_asset = account_info['assets'][idx]
#     idx = idx + 1    
#     if (scrutinized_asset['asset-id'] == asset_id):
#         holding = True
#         break
account_info

{'address': '2UZFZ27QS7VTVAAJ77UTCB7S5JKA6YWAICLKSPRIUULK7U2IRSNCN3VVA4',
 'amount': 5000000,
 'amount-without-pending-rewards': 5000000,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0},
 'assets': [],
 'created-apps': [],
 'created-assets': [],
 'min-balance': 100000,
 'pending-rewards': 0,
 'reward-base': 27521,
 'rewards': 0,
 'round': 27404942,
 'status': 'Offline',
 'total-apps-opted-in': 0,
 'total-assets-opted-in': 0,
 'total-created-apps': 0,
 'total-created-assets': 0}

In [105]:
m

'soap earth page uncle step caught whisper seed tissue summer novel base robot vacuum elder tourist staff tank just nerve ocean start table abandon scan'

In [143]:
receiver_pk

'2UZFZ27QS7VTVAAJ77UTCB7S5JKA6YWAICLKSPRIUULK7U2IRSNCN3VVA4'

In [ ]:
create_asset(m,receiver_pk)

In [119]:
asset_id = "156589497"

In [124]:
receiver

'speak steel layer waste elite novel easily pull believe casino three loan hurry abuse dinner junk winner jewel need resemble mixture glance change ability arch'

In [126]:
opt_in(receiver,asset_id)

AlgodHTTPError: 

In [ ]:
transfer_asset(m,receiver_pk,asset_id)